# Activation function 비교 실험

hidden layer의 activation function이 hypothesis의 형태를 어떻게 변화시키는지 확인하기 위한 예제.
어떤 과목 수강생의 학습 시간과 pass/fail여부 데이터를 학습하여 학습시간에 따른 pass/fail 예측 모형을 만든다.

activation function로 ReLU, sigmoid, tanh를 사용하여 그 예측 성능을 비교하고
만들어진 모델의 hypothesis를 2차원 시각화 하여 activation fuction에 따라 decision surface가 어떻게 형성되는지를 살펴보고자 한다.

In [ ]:
import tensorflow as tf
import numpy as np

In [ ]:
data = np.array([[2., 0.], [3., 0.], [4., 0.], [5., 1.], [7., 0.], [8., 1.], [9., 1.]], dtype=np.float32)
x = data[:,0]
y = data[:,1]

In [ ]:
def xavier_init(n_inputs, n_outputs, uniform=True):
    if uniform:
        init_range = tf.sqrt(6.0 / (n_inputs + n_outputs))
        return tf.random_uniform_initializer(-init_range, init_range)
    else:
        stddev = tf.sqrt(3.0 / (n_inputs + n_outputs))
        return tf.truncated_normal_initializer(stddev = stddev)

In [ ]:
l1_in = 1
l1_out = 4
l2_out = 4
l3_out = 1
init_min = -1.0
init_max = 1.0

with tf.name_scope('model'):
    X = tf.placeholder(dtype=tf.float32, name = 'input')
    Y = tf.placeholder(dtype=tf.float32, name = 'target')
    with tf.name_scope('hidden1'):
        i1 = tf.reshape(X, [1,-1])
        W1 = tf.get_variable("w_l1", shape=[l1_out, l1_in], initializer = xavier_init(l1_in, l1_out))
        b1 = tf.get_variable("b_l1", shape=[l1_out, ], initializer = xavier_init(l1_out, l1_out))
        #W1 = tf.Variable(tf.random_uniform([l1_fs, 1], init_min, init_max), dtype=tf.float32, name = 'w_l1')
        #b1 = tf.Variable(tf.random_uniform([l1_fs,], init_min, init_max), dtype=tf.float32, name = 'b_l1')
        b1 = tf.reshape(b1, [l1_out, -1])
        #l1 = tf.nn.relu(tf.matmul(W1, i1) + b1, name = 'o_l1')
        #l1 = tf.nn.sigmoid(tf.matmul(W1, i1) + b1, name = 'o_l1')
        l1 = tf.nn.tanh(tf.matmul(W1, i1) + b1, name = 'o_l1')
    with tf.name_scope('hidden2'):
        W2 = tf.get_variable("w_l2", shape=[l2_out, l1_out], initializer = xavier_init(l1_out, l2_out))
        b2 = tf.get_variable("b_l2", shape=[l2_out, ], initializer = xavier_init(l2_out, l2_out))
        #W2 = tf.Variable(tf.random_uniform([1, l1_fs], init_min, init_max), dtype=tf.float32, name = 'w_l2')
        #b2 = tf.Variable(tf.random_uniform([1], init_min, init_max), dtype=tf.float32, name = 'b_l2')
        b2 = tf.reshape(b2, [l2_out, -1])
        #l2 = tf.nn.relu(tf.matmul(W2, l1) + b2, name = 'o_l2')
        #l2 = tf.nn.sigmoid(tf.matmul(W2, l1) + b2, name = 'o_l2')
        l2 = tf.nn.tanh(tf.matmul(W2, l1) + b2, name = 'o_l2')
    with tf.name_scope('hidden3'):
        W3 = tf.get_variable("w_l3", shape=[l3_out, l2_out], initializer = xavier_init(l2_out, l3_out))
        b3 = tf.get_variable("b_l3", shape=[l3_out, ], initializer = xavier_init(l3_out, l3_out))
        l3 = tf.matmul(W3, l2) + b3
    with tf.name_scope('output'):
        output = tf.nn.sigmoid(l3)
        predict = tf.round(output)

In [ ]:
with tf.name_scope('eval'):
    loss = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits = l3, labels = Y, name = 'loss'))
    #loss = -tf.reduce_mean(tf.reduce_sum(Y * tf.log(y_) + (1-Y) * tf.log(1-y_)))
    opt = tf.train.GradientDescentOptimizer(learning_rate=0.1)
    train = opt.minimize(loss)
    tf.summary.scalar('loss', loss)

In [ ]:
%rm -rf log_dir

In [ ]:
writer = tf.summary.FileWriter("log_dir")
summary_op = tf.summary.merge_all()

sess = tf.InteractiveSession()
sess.run(tf.global_variables_initializer())
writer.add_graph(sess.graph)

In [ ]:
for step in range(50000):
    summary_val, loss_val, _ = sess.run([summary_op, loss, train], feed_dict={X:x, Y:y})
    '''
    w1_val, b1_val, l1_val, w2_val, b2_val, l2_val, summary_val, loss_val, _ = sess.run([W1, b1, l1, W2, b2, l2, summary_op, loss, train], feed_dict={X:x, Y:y})
    print(w1_val)
    print(b1_val)
    print(l1_val)
    print(w2_val)
    print(b2_val)
    print('l2')
    print(l2_val)
    print('true_value')
    print(y)
    print(loss_val)
    print('---')
    '''
    
    writer.add_summary(summary_val, step)
    if (step % 1000 == 0):
        print('step:', step, 'loss:', loss_val)
w1_val, b1_val, w2_val, b2_val = sess.run([W1, b1, W2, b2])
print('w1', w1_val, 'b1', b1_val, 'w2', w2_val, 'b2', b2_val)

In [ ]:
print(sess.run(output, feed_dict={X:[2.0, 3.0, 5.0, 7.0, 8.0]}))
print(sess.run(predict, feed_dict={X:[2.0, 3.0, 5.0, 7.0, 8.0]}))

In [ ]:
sess.close()

In [ ]:
#!tensorboard --log=log_dir

In [ ]:
tf.reset_default_graph()